# **新垣、有村、本田画像判別機**

## ライブラリのimport

In [0]:
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import os

In [0]:
import keras
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, list_pictures, load_img
from sklearn.model_selection import train_test_split

## driveと連携

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# 4/AABNGuqf5uPW-_ysCYL0derXJYBw2HbApAS1XcLZUU8i8JwRsIezfio

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive/"Colab Notebooks"/imgs

## 画像を読み込んでnumpy配列にコンバートする関数

In [0]:
def load_pic(data_path):
    # goodファイルのパスの読み込み
    path_list = glob.glob('drive/Colab Notebooks/imgs/' + data_path + '/*')
    # 画像読み込み用のリストの用意
    file_list = [None for i in range(len(path_list))]

    # 画像の読み込み
    for i in range(len(path_list)):
        file_list[i] = cv2.imread(path_list[i])
        #bgrからrgbへの変換(openCVでは一般的にBGR)  
        file_list[i] = cv2.cvtColor(file_list[i], cv2.COLOR_BGR2RGB)
        # size を 56*56にサイズ変更
        file_list[i] = cv2.resize(file_list[i], (56, 56))
    file_np = np.array(file_list)

    # ランダム数列を作成
    indices = np.arange(len(path_list))
    np.random.shuffle(indices)
    # 配列の順番をシャッフル
    file_list = file_np[indices]
    
    return file_list

## pltの際のグリッド線の削除

In [0]:
# clear label
def clearLabel(_ax):
  _ax.tick_params(labelbottom="off",bottom="off")
  _ax.tick_params(labelleft="off",left="off")
  _ax.set_xticklabels([]) 
  _ax.axis('off')
  return _ax

In [0]:
#ガッキーの読み込み
aragaki = load_pic("aragaki")
fig, ax = plt.subplots(dpi=30)
clearLabel(ax)
plt.imshow(aragaki[0])
plt.show()

In [0]:
#有村架純の読み込み
arimura = load_pic("arimura")
fig, ax = plt.subplots(dpi=30)
clearLabel(ax)
plt.imshow(arimura[0])
plt.show()

In [0]:
#本田翼の読み込み
honda = load_pic("honda")
fig, ax = plt.subplots(dpi=30)
clearLabel(ax)
plt.imshow(honda[0])
plt.show()

In [0]:
# aragaki = load_pic("aragaki", aragaki)
# arimura = load_pic("arimura", arimura)
# honda = load_pic("honda", honda)

## リストの結合とラベリング

In [0]:
# marge
X = np.concatenate([aragaki,arimura,honda])

#ラベル作成
Y = np.zeros([len(X)])
Y[len(aragaki):] = 1
Y[len(aragaki)+len(arimura):] = 2

## 正規化とOne-Hot 形式への変更(ex, 0-> [1,0,0], 2 -> [0,0,1])

In [0]:
# 画素値を0から1の範囲に変換
X_ = X.astype('float32')
X_ = X_ / 255.0

# クラスの形式を変換
Y_ = np_utils.to_categorical(Y, 3)

# 学習用データとテストデータ
X_train, X_test, Y_train, Y_test = train_test_split(X_, Y_, test_size=0.33, random_state=111)

## モデルの構築

## 各パラメータ

In [0]:
#何回回すかの決定
MAX_EPOCH = 1_500
# サンプリングの数を設定
BATCH_SIZE = 30

## モデルの構築2

In [0]:
# CNNを構築
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))       # クラスは3個
model.add(Activation('softmax'))

## モデルのコンパイル

In [0]:
# optimizer には adam を指定
adam = keras.optimizers.Adam(lr=1e-3)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

## 実行

In [0]:
history = model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=MAX_EPOCH,
                   validation_data = (X_test, Y_test), verbose = 1)

## 結果をプロット

In [0]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['acc', 'val_acc'], loc='lower right')
plt.show()

# 予測

In [0]:
# predict = model.predict( data_list )

In [0]:
# カテゴリの作成
category = ["新垣結衣","有村架純","本田翼"]

# random関数の作成
import random
def randam_data(data_set, label, batch_size) :
    index = np.random.choice(len(data_set),batch_size)
    data_set = data_set[index]
    label = label[index]
    return data_set, label

## 予測

In [0]:
test_x, test_y = randam_data(X_test, Y_test, 5)
pre_ram = model.predict(test_x)

In [0]:
for (i, pre) in enumerate(pre_ram):
  y = pre.argmax()
  y_ = test_y[i].argmax()
  print("予測 :{0}  {1}% ".format(category[y], round(pre[y]*100)) )
  print("正解: ", category[y_])
  fix, ax = plt.subplots(dpi=30)
  clearLabel(ax)
  plt.imshow(test_x[i])
  plt.show()

## 任意のファイルで予測テスト

## ファイルのアップロード

In [0]:
from google.colab import files
uploaded = files.upload()
!ls

In [0]:
img = cv2.imread("face2.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_ = cv2.resize(img, (56, 56))
img_ = [img_ , img_]
img_ = np.array(img_)

img_= img_.astype('float32')
img_ = img_ / 255.0

predict = model.predict(img_)
pre = predict[0]
y = pre.argmax()
print("予測 :{0}  {1}% ".format(category[y], round(pre[y]*100)) )
print("新垣結衣: {0}%, 有村架純: {1}%, 本田翼: {2}%".format(round(pre[0]*100),round(pre[1]*100),round(pre[2]*100)))
fix, ax = plt.subplots(dpi=30)
clearLabel(ax)
plt.imshow(img_[1])
plt.show()